In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,time,pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from utils import data_process, visualize
from utils.utils import train, test, summary, setup_seed
from utils.MCNN import MCNN
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [2]:
data = np.load("./data/Bureau_train.npy", allow_pickle=True)
x_train = data.item().get("x_train")
y_train = data.item().get("y_train")
data = np.load("./data/Bureau_test.npy", allow_pickle=True)
x_test = data.item().get("x_test")
y_test = data.item().get("y_test")
x_train=x_train.transpose((0,2,1))
x_test=x_test.transpose((0,2,1))

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()

x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()
traindata = TensorDataset(x_train, y_train)
testdata = TensorDataset(x_test, y_test)
train_loader = DataLoader(traindata, batch_size=256, shuffle=True)
test_loader = DataLoader(testdata, batch_size=256, shuffle=True)

In [3]:
%%time
setup_seed(42)
model = MCNN().cuda()
L = []
test_hist = []
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 200 + 1):
    start = time.time()
    loss = train(model, train_loader, optimizer, epoch)
    L.append(loss)
    print("time {:.1f} sec:".format(time.time() - start))
    acc = test(model, test_loader)
    test_hist.append(acc)

AssertionError: Torch not compiled with CUDA enabled

In [67]:
model.eval()
y_pred = []
out=[]
y_test = []
with torch.no_grad():
    for data, target in test_loader:
        data = data.cuda()
        output = model(data)
        output=F.softmax(output, dim=1)
        pred = output.max(1, keepdim=True)[1]  
        pred = pred.cpu().numpy().squeeze()
        y_pred.append(pred)
        out.append(output.cpu().numpy())
        y_test.append(target.numpy().squeeze())
y_pred=np.hstack(y_pred)
y_test=np.hstack(y_test)
out=np.vstack(out)

summary(y_pred,y_test)

confusion matrix:
 [[818   0]
 [  0 417]]
Roc score: 1.0000
F1 score: 1.0000
Recall_score: 1.0000
Precision_score: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       818
           1       1.00      1.00      1.00       417

    accuracy                           1.00      1235
   macro avg       1.00      1.00      1.00      1235
weighted avg       1.00      1.00      1.00      1235



In [8]:
earth=np.where(y_pred==1)[0]
non_earth=np.where(y_pred==0)[0]

p_earth=out[earth,1].squeeze()

p_non_earth=out[non_earth,0].squeeze()

In [9]:
a = np.random.rand(100)
hist,bins = np.histogram(p_earth,bins=10,range=(0,1))
print(hist)
print(bins)


[  0   0   0   0   0   4   1   2   4 233]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [10]:
a = np.random.rand(100)
hist,bins = np.histogram(p_non_earth,bins=10,range=(0,1))
print(hist)
print(bins)

[   0    0    0    0    0    1    1    1    0 1823]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
